In [1]:
import pandas as pd
import requests
from lxml import html
import time

In [135]:
words = pd.read_excel('Words from GRE Vocabulary Assessment Videos.xlsx',header=None)

In [3]:
set1 = words['Group 1'][1:31]

In [144]:
def get_my_set(data, set_no):
    '''
    Parameters: 
    
    data:- gregmat excel sheet dataframe
    set_no:- int, set no. which you want to get
    
    Returns: Series of 30 words
    '''
    column_name = 'Group '+ str(set_no)
    columns = data.columns
    for col in columns:
        if data[col].astype(str).str.fullmatch(column_name).any():
            found = True
            match_col = col
            for index, row in enumerate(data[col]):
                if row == column_name:
                    match_row = index
        
    selected = data[match_col][match_row+2:32]
    return selected.reset_index(drop=True)

In [147]:
get_my_set(words, 3)

0            abate
1           abjure
2        anomalous
3       antipathy 
4           arcane
5          arduous
6          artless
7          ascetic
8         assuage 
9           betray
10        bucolic 
11         burgeon
12    cacophonous 
13        canonize
14         censure
15      chicanery 
16        coalesce
17          cogent
18      compelling
19         contend
20        copious 
21    cosmopolitan
22       deference
23      desultory 
24       diffident
25       dilatory 
26      equivocate
27        polarize
28        prodigal
29         verbose
Name: 4, dtype: object

In [113]:
column_name = 'Group 1'
data = words

columns = words.columns
for col in columns:
#     print(col)
    print(data[data[col].astype(str).str.match(column_name).any()])

0                         NaN
1                         NaN
2                         NaN
3                         NaN
4     TEST 28 HAS ARRIVEDDDDD
               ...           
93                        NaN
94                        NaN
95                        NaN
96                        NaN
97                        NaN
Name: 1, Length: 98, dtype: object
0         Group 1
1     Take Test 1
2          abound
3       amorphous
4         austere
         ...     
93     repertoire
94      resilient
95       resolute
96         supple
97          valor
Name: 0, Length: 98, dtype: object
0                         NaN
1                         NaN
2                         NaN
3                         NaN
4     TEST 28 HAS ARRIVEDDDDD
               ...           
93                        NaN
94                        NaN
95                        NaN
96                        NaN
97                        NaN
Name: 1, Length: 98, dtype: object
0         Group 1
1     Take Test 1
2

In [21]:
def get_def_mnemo(word):
    url = 'https://mnemonicdictionary.com/?word='+word

    page = requests.get(url)
    tree = html.fromstring(page.content)
    definition = tree.xpath('//li[@class="media list-group-item p-4"]/div[1]//text()')[12].strip()
    
#     print("\n\nWord: \t\t{}\nDefinition: \t{}".format(word, definition))
    
    count=1
    mnemonics = []
    for index, mnemonic_id in enumerate(range(0,7)):
        
        try:
            mnemonic = tree.xpath('//div[@class="card-text"]/p//text()')[mnemonic_id].strip()
        except:
            continue
        if mnemonic in ['','Powered by','Mnemonic Dictionary']:
            continue
        
        if count > 3:
            break
            
#         print("Mnemonic {}: \t{}".format(count, mnemonic))
        mnemonics.append(mnemonic)
        count+=1
    return definition, mnemonics

In [ ]:
def get_image(word):
    image_url = '{}{}{}'.format('https://dailyvocab.com/photos/', word, '/')
    page = requests.get(image_url)
    tree = html.fromstring(page.content)
    image_url = tree.xpath('//meta[@name ="twitter:image"]/@content')
    return image_url

In [85]:
ankified = pd.DataFrame(columns=['words','definition','mnemonics','image_url'])

In [88]:
for word in set1:
    if word.lower() in list( ankified['words'].apply( lambda x:x.lower()) ):
        continue
    definition, mnemonics = get_def_mnemo(word)
    image_url = get_image(word)
    print(word)
    print('Definition: {}'.format(definition))
    for index, mnemo in enumerate(mnemonics):
        print('Mnemonic {}: {}'.format(index+1, mnemo))
    print("Image url: {}".format(image_url))
    print('#'*125)
    time.sleep(2)
    ankified = ankified.append({'words':word,'definition':definition,'mnemonics':mnemonics,'image_url':image_url}, ignore_index=True)

In [87]:
ankified

,words,definition,mnemonics,image_url
0,abound,(verb) be abundant or plentiful; exist in larg...,[Bounded means limited. Abound means not limit...,[]
1,amorphous,(adj) having no definite form or distinct shape,"[In Greek 'morphe' means shape/form.\r\nSo, am...",[]
2,austere,(adj) severely simple,[AUS-australia TERE-tear(sorrow). like the rec...,[]
3,belie,(verb) be in contradiction with,"[belie has lie., belie ~ lie means something f...",[]
4,capricious,(adj) changeable,[ca(CAR) + PRIC(PRICE)...PRICE OF cars nowaday...,[]
5,cerebral,(adj) involving intelligence rather than emoti...,"[cereBRAINal, CEREBRAL ASSASIN]",[]
6,congenial,(adj) suitable to your needs,"[Genelia Desouza was suitable, appropriate ,c...",[https://dailyvocab.com/wp-content/uploads/201...
7,conspicuous,(adj) obvious to the eye or mind,[Con(can)+ s(see) + the + pic(picture) --> can...,[]
8,cursory,(adj) hasty and without attention to detail; n...,[relate it to the cursor on your computer scre...,[]
9,daunting,(adj) discouraging through fear,[DAUN SOUNDS LIKE DON\r\nNTING SOUNDS LIKE ACT...,[https://dailyvocab.com/wp-content/uploads/201...


# Requests and BS4

In [29]:
page = requests.get(image_url)
page.status_code

200

In [30]:
from bs4 import BeautifulSoup

In [31]:
soup = BeautifulSoup(page.text, 'html.parser')

In [32]:
soup.find_all('p')

[<p class="site-title">
 <a href="https://dailyvocab.com/">
 
 </a>
 </p>,
 <p class="site-title">
 <a href="https://dailyvocab.com/">
 
 </a>
 </p>,
 <p><strong>Obsequious [<span class="dbox-italic">uh</span> b-<span class="dbox-bold">see</span>-kwee-<span class="dbox-italic">uh</span> s]</strong></p>,
 <p><strong>Obsequious (अब्सीक्वीअस) meaning in Hindi (English to Hindi Dictionary)</strong></p>,
 <p><strong>Obsequious in a sentence</strong></p>,
 <p><strong>Mnemonic trick to remember the meaning of Obsequious</strong></p>,
 <p><strong>Obsequious Pronunciation</strong></p>,
 <p><div class="rll-youtube-player" data-id="JtFc8lcXt40" data-query="" data-src="https://www.youtube.com/embed/JtFc8lcXt40"></div><noscript><iframe allowfullscreen="allowfullscreen" frameborder="0" height="315" src="https://www.youtube.com/embed/JtFc8lcXt40" width="420"></iframe></noscript></p>,
 <p><strong>The Great Indian Chamcha</strong></p>,
 <p><div class="rll-youtube-player" data-id="DHfR04Jm54g" data-quer

In [33]:
soup.find_all('p')[2].get_text()

'Obsequious\xa0[uh b-see-kwee-uh s]'

In [34]:
print(soup.prettify())

<!DOCTYPE HTML>
<html lang="en-US" xmlns:fb="http://ogp.me/ns/fb#">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
  <style id="kirki-css-vars">
   :root{}
  </style>
  <script type="text/javascript">
   (function( $ ) {

			var themeModeStylesheetURL = 'https://dailyvocab.com/wp-content/themes/boombox/css/day-mode.min.css';
			var getCookie = function( cname ) {
				var name = cname + "=";
				var decodedCookie = decodeURIComponent( document.cookie );
				var ca = decodedCookie.split( ';' );
				for ( var i = 0; i < ca.length; i++ ) {
					var c = ca[i];
					while ( c.charAt( 0 ) == ' ' ) {
						c = c.substring( 1 );
					}
					if ( c.indexOf( name ) == 0 ) {
						return c.substring( name.length, c.length );
					}
				}
				return "";
			};
			var setCookie = function( cname, cvalue, exdays ) {
				var d = new Date();
				d.setTime( d.getTime() + (exdays * 24 * 60 * 60 * 1000) );
			